In [2]:
# fake_profile_detection

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout,
    Conv2D, MaxPooling2D, Flatten, concatenate
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# ----------------------------
# 1. Load Instagram Dataset (CSV)
# ----------------------------
df = pd.read_csv("Instagram_fake_profile_dataset.csv")  # Update with your file path

# Inspect columns to determine relevant ones
print("Columns in dataset:", df.columns)

# Example feature columns (update as actual dataset columns)
feature_cols = [
    "followers_count", "following_count",
    "posts_count", "bio_length", "engagement_rate"
]
X_meta = df[feature_cols].values
y = df["is_fake"].values  # Assuming 'is_fake' is the label column

# If there are no profile images, use synthetic placeholder images
# (Or load real ones if available)
X_images = np.random.rand(len(df), 32, 32, 3)

# ----------------------------
# 2. Split & Preprocess
# ----------------------------
X_meta_train, X_meta_test, X_img_train, X_img_test, y_train, y_test = train_test_split(
    X_meta, X_images, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_meta_train = scaler.fit_transform(X_meta_train)
X_meta_test = scaler.transform(X_meta_test)

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

# ----------------------------
# 3. Build ANN for Metadata
# ----------------------------
meta_input = Input(shape=(X_meta_train.shape[1],), name="metadata_input")
m = Dense(32, activation="relu")(meta_input)
m = Dropout(0.3)(m)
m = Dense(16, activation="relu")(m)
meta_output = Dense(8, activation="relu")(m)

# ----------------------------
# 4. Build CNN for Images
# ----------------------------
img_input = Input(shape=(32, 32, 3), name="image_input")
i = Conv2D(32, (3, 3), activation="relu")(img_input)
i = MaxPooling2D((2, 2))(i)
i = Conv2D(64, (3, 3), activation="relu")(i)
i = MaxPooling2D((2, 2))(i)
i = Flatten()(i)
i = Dense(64, activation="relu")(i)
img_output = Dense(32, activation="relu")(i)

# ----------------------------
# 5. Combine Models & Compile
# ----------------------------
combined = concatenate([meta_output, img_output])
c = Dense(32, activation="relu")(combined)
c = Dropout(0.4)(c)
c = Dense(16, activation="relu")(c)
final_output = Dense(2, activation="softmax", name="output")(c)

model = Model(inputs=[meta_input, img_input], outputs=final_output)
model.compile(optimizer=Adam(0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# ----------------------------
# 6. Train
# ----------------------------
history = model.fit(
    [X_meta_train, X_img_train], y_train,
    validation_data=([X_meta_test, X_img_test], y_test),
    epochs=10,
    batch_size=32,
    verbose=1
)

# ----------------------------
# 7. Evaluate
# ----------------------------
loss, acc = model.evaluate([X_meta_test, X_img_test], y_test, verbose=0)
print(f" Test Accuracy: {acc*100:.2f}%")

# ----------------------------
# 8. Save Model
# ----------------------------
model.save("fake_profile_detector_instagram.h5")
print(" Saved as fake_profile_detector_instagram.h5")

ModuleNotFoundError: No module named 'tensorflow'